In [18]:
import os

directory = r'D:\github\Cricket-Prediction\data\3_aftermerging'

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("filtering").getOrCreate()

team12Stats = spark.read.csv(os.path.join(directory, 'team12Statsflip.csv'), header=True, inferSchema=True)
balltoball = spark.read.csv(os.path.join(directory, 'balltoballflip.csv'), header=True, inferSchema=True)
playersStats = spark.read.csv(os.path.join(directory, 'playersStatsflip.csv'), header=True, inferSchema=True)

In [19]:
team12Stats.select('match_id').distinct().count(), balltoball.select('match_id').distinct().count(), playersStats.select('match_id').distinct().count()

(3399, 3683, 1200)

In [20]:
# Extract match_id columns from each dataset
team12_match_ids = team12Stats.select('match_id').distinct()
balltoball_match_ids = balltoball.select('match_id').distinct()
player_match_ids = playersStats.select('match_id').distinct()

# Find intersection of match_id across all three datasets
common_match_ids = team12_match_ids.intersect(balltoball_match_ids).intersect(player_match_ids)
common_match_ids.collect()

# convert to list
common_match_ids_list = [row['match_id'] for row in common_match_ids.collect()]

print(common_match_ids_list),len(common_match_ids_list)

[1031433, 1321259, 1310175, 1187667, 446960, 1072317, 1263151, 1197521, 1348324, 1383095, 1298157, 412686, 1321311, 225263, 1317908, 1310179, 722335, 702141, 1317910, 412683, 1215138, 1233974, 1034829, 1444951, 1273730, 1444948, 1286684, 1229822, 1199543, 1263152, 412682, 1072321, 1389395, 1447482, 951331, 1415751, 635658, 1275277, 660223, 1185315, 1317907, 1020029, 951305, 1187668, 1339605, 1399118, 1381458, 1384594, 1424832, 660113, 1215165, 1362242, 1140384, 1072318, 1257945, 298804, 1089202, 540174, 567205, 1436481, 518955, 1270838, 875491, 287866, 1041615, 1276913, 412691, 1286672, 1321308, 258463, 1273721, 682965, 623572, 592269, 1243388, 287865, 1263157, 848839, 1144173, 1197407, 754717, 1424763, 306987, 1327272, 915785, 452154, 1273149, 387563, 1273727, 1343780, 1275274, 287857, 423782, 1321270, 592268, 1283043, 1321306, 1381457, 1074957, 1279379, 565820, 636536, 660209, 1404391, 1273750, 287864, 489212, 1428151, 1405314, 1273737, 1278672, 1140992, 1263710, 1298172, 1321281, 12

(None, 1156)

In [21]:
# filter team12Stats

filtered_team12Stats = team12Stats.filter(team12Stats.match_id.isin(common_match_ids_list)).drop("_c0")

# filter balltoball

filtered_balltoball = balltoball.filter(balltoball.match_id.isin(common_match_ids_list)).drop("_c0")

# filter playersStats

filtered_playersStats = playersStats.filter(playersStats.match_id.isin(common_match_ids_list)).drop("_c0")

In [22]:
print((filtered_playersStats.count(), len(filtered_playersStats.columns)), 
(filtered_team12Stats.count(), len(filtered_team12Stats.columns)), 
(filtered_balltoball.select('match_id', 'flip').distinct().count(), len(filtered_balltoball.columns)))

(50864, 15) (2312, 17) (2312, 7)


In [23]:
filtered_team12Stats.show(5)

+--------+----+------+--------------------+---------------------+---------------------+-------------------+--------------------+-----------------------+-----------------------+--------------------+---------------------+---------------------+-------------------+--------------------+-----------------------+-----------------------+
|match_id|flip|gender|Cumulative Won team1|Cumulative Lost team1|Cumulative Tied team1|Cumulative NR team1|Cumulative W/L team1|Cumulative AveRPW team1|Cumulative AveRPO team1|Cumulative Won team2|Cumulative Lost team2|Cumulative Tied team2|Cumulative NR team2|Cumulative W/L team2|Cumulative AveRPW team2|Cumulative AveRPO team2|
+--------+----+------+--------------------+---------------------+---------------------+-------------------+--------------------+-----------------------+-----------------------+--------------------+---------------------+---------------------+-------------------+--------------------+-----------------------+-----------------------+
|  2110

In [24]:
# Save filtered datasets by coner to pandas
filtered_team12Stats.toPandas().to_csv(os.path.join(directory, '../4_filteredData/team12Stats.csv'), index=False)
filtered_balltoball.toPandas().to_csv(os.path.join(directory, '../4_filteredData/balltoball.csv'), index=False)
filtered_playersStats.toPandas().to_csv(os.path.join(directory, '../4_filteredData/playersStats.csv'), index=False)